In [283]:
import pandas as pd
import numpy as np
import pickle
from Bio import SeqIO
from Bio import Phylo
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import zlib

# Remove recombinations from MSA

+ input: MSA (fa, xmfa), tree, predicted importations 
+ output 1: MSA without recombination
    + fasta
    + phylip
+ output 2: table of protein ids assigned to importations 


# Results for the test sets

## MSA lengths
+ rso-test (fjat) = 3,825,198 (xmfa: 7,666,198)
+ phy2 = 2,987,190 (xmfa: 6,086,190)
+ phy4 = 3,219,942
+ phy3 = 3,459,843 (xmfa: 6,947,843)

(for ASM359060v in phy2 dataset the summed length is 116,425)

## Summed recombinations length
+ rso = 92,925
+ phy2 = 290,218
+ phy4 = 58,063
+ phy3 = 120,259

(phy4 strain PSI07 is from Indonesia! very different)

In [284]:
DEBUG = False
SPlIT = True

if DEBUG:
    SAMPLE = 'phy2_test' #'korea2' 'rso_test' 'phy2_test'

    recombis = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_cfml/rso_test_cfml.importation_status.txt'
    path2tree = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_cfml/rso_test_cfml.labelled_tree.newick'
    xm_msa_in = "/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_concat.xmfa"
    fa_msa_in = "/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_concat.fasta"
    accDB = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/MSA_ACC_DB_rso_test.pkl'
    protDB = '../'+SAMPLE+'_outputs/GENOME_PROTEIN_DB_'+SAMPLE+'.pkl'
    # out
    fa_msa_out = "/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_concat_no_RR.fasta"
    updated_recombis = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_recombis2prot.csv'
    mult_recombis = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_recombis_mult_genes.txt'
    top_recombis = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/rso_test_outputs/rso_test_recombis_top_genes.txt'

    # in
    recombis = recombis.replace("rso_test", SAMPLE)
    path2tree = path2tree.replace("rso_test", SAMPLE)
    xm_msa_in = xm_msa_in.replace("rso_test", SAMPLE)
    fa_msa_in = fa_msa_in.replace("rso_test", SAMPLE)
    accDB = accDB.replace("rso_test", SAMPLE)
    protDB = protDB.replace("rso_test", SAMPLE)
    # out
    fa_msa_out = fa_msa_out.replace("rso_test", SAMPLE)
    updated_recombis = updated_recombis.replace("rso_test", SAMPLE)
    mult_recombis = mult_recombis.replace("rso_test", SAMPLE)
    top_recombis = top_recombis.replace("rso_test", SAMPLE)
else:
    # in
    recombis = snakemake.params["recombis"]
    path2tree = snakemake.params["tree"]
    xm_msa_in = snakemake.input["xm_msa_in"]
    fa_msa_in = snakemake.input["fa_msa_in"]
    accDB = snakemake.input["accDB"]
    protDB = snakemake.input["protDB"]
    # out
    fa_msa_out = snakemake.output["fa_msa_out"]
    updated_recombis = snakemake.output["updated_recombis"]
    mult_recombis = snakemake.output["mult_recombis"]
    top_recombis = snakemake.output["top_recombis"]

In [285]:
if SPlIT:
    xm_msa_in = xm_msa_in.replace('.xmfa', '_s1.xmfa')
    fa_msa_out = fa_msa_out.replace('_no_RR.fasta', '_TMP.fasta')

In [286]:
xm_msa_in

'/Users/devseeva/Desktop/work/sm_workflow/geneTrack/phy2_test_outputs/phy2_test_concat_s1.xmfa'

# Replace the inner nodes with the leaves 

In [287]:
imp_stat = pd.read_csv(recombis, sep='\t')
imp_stat

,Node,Beg,End
0,IBSBF1503,403132,403427
1,RS_488,30547,30775
2,RS_488,388081,388084
3,RS_488,2128141,2128147
4,RS_488,2620391,2620496
...,...,...,...
647,NODE_15,2469493,2469703
648,NODE_15,2526073,2526296
649,NODE_15,2683079,2684072
650,NODE_15,2742577,2742809


In [288]:
for tree in Phylo.parse(path2tree, 'newick'):
    parents = {}
    # iterate through inner nodes 
    for clade in tree.get_nonterminals(order="level"):
        parents[clade.name] = []
        # get leaves for each inner node
        for child in clade.get_terminals():
            parents[clade.name].append(child.name)
        #parents[clade.name] = ";".join(parents[clade.name])
parents

{'NODE_17': ['Po82',
  'CIAT_078',
  'IBSBF1503',
  'UW163',
  'IBSBF_2571',
  'RS_489',
  'UW576',
  'RS_488',
  'UY031'],
 'NODE_16': ['Po82',
  'CIAT_078',
  'IBSBF1503',
  'UW163',
  'IBSBF_2571',
  'RS_489',
  'UW576',
  'RS_488'],
 'NODE_15': ['Po82',
  'CIAT_078',
  'IBSBF1503',
  'UW163',
  'IBSBF_2571',
  'RS_489',
  'UW576'],
 'NODE_14': ['Po82', 'CIAT_078', 'IBSBF1503', 'UW163', 'IBSBF_2571'],
 'NODE_13': ['RS_489', 'UW576'],
 'NODE_12': ['Po82', 'CIAT_078', 'IBSBF1503', 'UW163'],
 'NODE_11': ['CIAT_078', 'IBSBF1503', 'UW163'],
 'NODE_10': ['IBSBF1503', 'UW163']}

In [289]:
new_len = len(imp_stat)

for p in parents:

    is_node = imp_stat["Node"].str.startswith(p)
    num_child = len(parents[p]) 
    print(p,'has',len(imp_stat[is_node]),'recombinations and',num_child,'children')
    new_len = new_len + len(imp_stat[is_node]) * (num_child - 1)
    
    #imp_stat = imp_stat.append([imp_stat[is_node]]*(num_child-1), ignore_index=True)
    print()
    
    # for each child: duplicate the parent row
    for child in parents[p]:       
        node_imp = imp_stat[is_node].replace({p:child}) 
        imp_stat = imp_stat.append([node_imp], ignore_index=True)
        is_node = imp_stat["Node"].str.startswith(p)

# remove the remaining parent rows

imp_stat = imp_stat.loc[~imp_stat["Node"].str.contains("NODE_")]
print('DF length after parent nodes expantion =', len(imp_stat), '; Must be =', new_len)
assert(new_len == len(imp_stat))

NODE_17 has 0 recombinations and 9 children

NODE_16 has 0 recombinations and 8 children

NODE_15 has 32 recombinations and 7 children

NODE_14 has 65 recombinations and 5 children

NODE_13 has 138 recombinations and 2 children

NODE_12 has 48 recombinations and 4 children

NODE_11 has 0 recombinations and 3 children

NODE_10 has 1 recombinations and 2 children

DF length after parent nodes expantion = 1387 ; Must be = 1387


# Sort and sum the importations

In [290]:
imp_stat = imp_stat.sort_values(by=['Beg', 'End'])
imp_stat

,Node,Beg,End
63,IBSBF_2571,18392,18528
1201,RS_489,24163,24407
1339,UW576,24163,24407
1202,RS_489,25741,25946
1340,UW576,25741,25946
...,...,...,...
264,IBSBF_2571,2924360,2925356
62,RS_489,2939510,2939648
1338,RS_489,2945986,2946091
1476,UW576,2945986,2946091


In [291]:
beg = imp_stat["Beg"].values.copy()
end = imp_stat["End"].values.copy()

In [292]:
ranges2del = []
for i in range(len(imp_stat)):
    ranges2del.extend(range(beg[i]-1, end[i]))
ranges2del = list(dict.fromkeys(ranges2del))
print('Summed recombination length:', len(ranges2del))

Summed recombination length: 228861


# Fasta vs. XMFA positions mapping list

* list of updated positions 
* index represent Fasta sequence positions 
* values represent XMFA sequence positions 

In [293]:
# parse the genes MSAs into vector of the alignments' lengths
aln_length = []
aln = open(xm_msa_in, 'r')

for l in aln:
    
    if l.startswith('>'):
        c = 0
    elif l.startswith('='):
        aln_length.append(c-1) # minus one char for the \n symbol
    else:
        c = c + len(l)
        
print('Found genes MSAs / LCBs:', len(aln_length))    
aln.close()

Found genes MSAs / LCBs: 1550


In [294]:
# insert 1000 count after each gene MSA
index_mapping = list(range(0,aln_length[0]))
sum_lengths = aln_length[0]

for i in range(1,len(aln_length)):  
    index_mapping.extend(range(sum_lengths + i*1000, sum_lengths + aln_length[i] + i*1000))
    sum_lengths = sum_lengths + aln_length[i]
    

In [295]:
print('Fasta length', sum(aln_length), '\nXMFA length', sum(aln_length)+(len(aln_length)-1)*1000)
# index = fasta file length
assert(sum(aln_length) == len(index_mapping))
# values = xmfa file length
assert(sum(aln_length)+(len(aln_length)-1)*1000 == index_mapping[-1]+1) 

Fasta length 1403790 
XMFA length 2952790


# Get the fasta position of each insertion from the xmfa position

+ use ranges to improve the running time of .index(i, start, end)

In [296]:
fa_beg = []
fa_end = []
previous_beg = 0
previous_end = 0
L=len(index_mapping)

for r_i in range(len(imp_stat)):
    
    # -1 for 0-start indexing
    r_beg = beg[r_i] - 1
    r_end = end[r_i] - 1
    #print(r_beg, r_end)
    
    # USE PREVIOUS_BEG FOR BOTH BECAUSE OF THE SORTING ISSUE
    r_beg_index = index_mapping.index(r_beg, previous_beg, L)#r_beg+1)
    r_end_index = index_mapping.index(r_end, previous_beg, L)#r_end+1)
    
    previous_beg = r_beg_index
    previous_end = r_end_index
    
    fa_beg.append(r_beg_index)
    fa_end.append(r_end_index)
    
    #print(r_i,"xmfa - fa:",r_beg_index, r_end_index)

imp_stat['FA Beg'] = fa_beg
imp_stat['FA End'] = fa_end

In [297]:
imp_stat['debug_len_must'] = imp_stat['End'] - imp_stat['Beg']
imp_stat["debug_len_is"] = imp_stat['FA End'] - imp_stat['FA Beg']
#assert(len(imp_stat.loc[imp_stat['debug_len_must'] != imp_stat['debug_len_is']]) == 0)
#imp_stat
imp_stat.loc[imp_stat['debug_len_must'] != imp_stat['debug_len_is']].to_csv(mult_recombis, index=False)

In [298]:
imp_stat.loc[imp_stat['debug_len_must'] != imp_stat['debug_len_is']]

,Node,Beg,End,FA Beg,FA End,debug_len_must,debug_len_is
70,IBSBF_2571,130056,131459,51055,51458,1403,403
1207,RS_489,378129,380152,141128,142151,2023,1023
1345,UW576,378129,380152,141128,142151,2023,1023
98,IBSBF_2571,742902,744169,283901,284168,1267,267
1225,RS_489,1008364,1010726,401363,402725,2362,1362
1363,UW576,1008364,1010726,401363,402725,2362,1362
291,UW576,1099168,1101414,447167,448413,2246,1246
889,Po82,1122457,1127983,459456,462982,5526,3526
954,CIAT_078,1122457,1127983,459456,462982,5526,3526
1019,IBSBF1503,1122457,1127983,459456,462982,5526,3526


# Delete the recombinations from the fasta file 

In [299]:
fasta_out = open(fa_msa_out, 'w')
num_of_strains = 0
for record in SeqIO.parse(fa_msa_in, 'fasta'):
    num_of_strains = num_of_strains + 1
    fasta_out.write('>' + record.id + '\n')
    genome_imp_stat = imp_stat.loc[imp_stat['Node'] == record.id]
    beg = genome_imp_stat["FA Beg"].values.copy()
    end = genome_imp_stat["FA End"].values.copy()
    
    ranges2del = []
    for i in range(len(genome_imp_stat)):
        ranges2del.extend(range(beg[i]-1, end[i]))
    ranges2del = list(dict.fromkeys(ranges2del))
    
    print('Genome',record.id,'has',len(ranges2del),'recombinations')
    #print(ranges2del)
    
    print('Number of gaps before',record.seq.count('-'))
    seq = list(record.seq)
    for to_del in ranges2del:
        seq[to_del] = '-'
    seq = ''.join(seq)
    print('Number of gaps after',seq.count('-'))
    print()
    fasta_out.write(seq + '\n')
    
fasta_out.close()

Genome Po82 has 46549 recombinations
Number of gaps before 13440
Number of gaps after 58800

Genome UY031 has 0 recombinations
Number of gaps before 11052
Number of gaps after 11052

Genome UW163 has 46605 recombinations
Number of gaps before 10200
Number of gaps after 56485

Genome IBSBF1503 has 46901 recombinations
Number of gaps before 10308
Number of gaps after 56843

Genome RS_488 has 346 recombinations
Number of gaps before 13607
Number of gaps after 13946

Genome RS_489 has 112440 recombinations
Number of gaps before 9559
Number of gaps after 121587

Genome IBSBF_2571 has 78406 recombinations
Number of gaps before 13701
Number of gaps after 90792

Genome CIAT_078 has 46794 recombinations
Number of gaps before 10770
Number of gaps after 57244

Genome UW576 has 118229 recombinations
Number of gaps before 13605
Number of gaps after 131272



# Expand and export the importation_status table

In [300]:
# read Accession Database

with open(accDB, 'rb') as handle:
    acc = pickle.load(handle)
acc#.keys()

['OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG0000210',
 'OG00

In [301]:
# read Protein Database

prot_db = pd.read_pickle(protDB)
prot_db = prot_db.rename(columns=lambda x: str(x).split('$')[0])
prot_db

,CIAT_078,UW163,RS_488,UW576,IBSBF1503,Po82,RS_489,IBSBF_2571,UY031
OG0000865,{'nucl': b'x\x9c5\x90\xd1\r\x001\x08Bg#|\xb8\x...,{'nucl': b'x\x9c5\x90\xcb\x11\x001\x08Bkc8\xd8...,"{'nucl': b""x\x9c-\x90\xc1\r\x00 \x0c\x02g#<\xb...",{'nucl': b'x\x9c5P\xc7\r\x001\x0c\x9a\r\xf1`\x...,{'nucl': b'x\x9c5\x90\xcb\x11\x001\x08Bkc8\xd8...,{'nucl': b'x\x9c5\x90\xd1\x11\xc00\x08Bg\xf3\x...,{'nucl': b'x\x9c-\x90A\x0eD1\x08B\xcfFX\xb8\xf...,{'nucl': b'x\x9c5\x90\xd1\x11\xc00\x08Bg\xf3\x...,"{'nucl': b""x\x9c-\x90\xc1\r\x00 \x0c\x02g#<\xb..."
OG0001813,{'nucl': b'x\x9c5\x93\xd9\x15\xc40\x08\x03k\xd...,{'nucl': b'x\x9c5\x93\xd9\x15\xc40\x08\x03k\xd...,"{'nucl': b""x\x9c=\x93Q\x12\xc3@\x08B\xcf\xe6\x...",{'nucl': b'x\x9c5\x93\xd1\x15\xc30\x08\x03g\xd...,{'nucl': b'x\x9c5\x93\xd9\x15\xc40\x08\x03k\xd...,{'nucl': b'x\x9c5\x93\xd9\x15\xc40\x08\x03k\xd...,"{'nucl': b""x\x9c=\x93Q\x12\xc3@\x08B\xcf\xe6\x...",{'nucl': b'x\x9c5\x93\xd9\x15\xc40\x08\x03k\xd...,"{'nucl': b""x\x9c=\x93Q\x12\xc3@\x08B\xcf\xe6\x..."
OG0001814,{'nucl': b'x\x9cMP\xdb\r\xc40\x0c\x9a\xcd\xe2\...,{'nucl': b'x\x9cMP\xdb\r\xc40\x0c\x9a\xcd\xe2\...,{'nucl': b'x\x9cMP\xdb\r\xc40\x0c\x9a\r\xf1\xc...,{'nucl': b'x\x9cM\x91\xd1\r\xc40\x0cBg\xb3\xf8...,{'nucl': b'x\x9cMP\xdb\r\xc40\x0c\x9a\xcd\xe2\...,{'nucl': b'x\x9cMP\xdb\r\xc40\x0c\x9a\xcd\xe2\...,{'nucl': b'x\x9c5\x90\xc1\x11\x04A\x08\x02c\xb...,{'nucl': b'x\x9cMP\xdb\r\xc40\x0c\x9a\xcd\xe2\...,{'nucl': b'x\x9cMP\xdb\r\xc40\x0c\x9a\r\xf1\xc...
OG0001815,"{'nucl': b""x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...","{'nucl': b""x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...",{'nucl': b'x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...,{'nucl': b'x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...,"{'nucl': b""x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...","{'nucl': b""x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...","{'nucl': b""x\x9c-\x90\xc1\r\x000\x08\x02g#<X\x...","{'nucl': b""x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...",{'nucl': b'x\x9c-\x90\xc1\r\x000\x08\x02g#>X\x...
OG0001816,{'nucl': b'x\x9cE\x92\xd1\x91\xc40\x0cBkc\xf8P...,{'nucl': b'x\x9cE\x92\xd1\x91\xc40\x0cBkc\xf8P...,{'nucl': b'x\x9cE\x92\xd9\x11\x031\x0cBkc\xf4\...,{'nucl': b'x\x9cM\x92\xd1\x91\xc40\x0cBkc\xf4\...,{'nucl': b'x\x9cE\x92\xd1\x91\xc40\x0cBkc\xf8P...,{'nucl': b'x\x9cE\x92\xd1\x91\xc40\x0cBkc\xf8P...,"{'nucl': b""x\x9cE\x92Kn\x04A\x0cB\xcf\x86Xx\x1...",{'nucl': b'x\x9cE\x92\xd1\x91\xc40\x0cBkc\xf8P...,{'nucl': b'x\x9cE\x92\xd9\x11\x031\x0cBkc\xf4\...
...,...,...,...,...,...,...,...,...,...
OG0002936,"{'nucl': b""x\x9c=\x91\xd9\x11\x04A\x08Bc\xb3\x...","{'nucl': b""x\x9c=\x91\xd9\x11\x04A\x08Bc\xb3\x...",{'nucl': b'x\x9c=Q\xc9\x11\x04A\x08\x8a\x8d\xe...,{'nucl': b'x\x9c=\x91\xc1\r\x041\x08\x03k\xb3x...,"{'nucl': b""x\x9c=\x91\xd9\x11\x04A\x08Bc\xb3\x...","{'nucl': b""x\x9c=\x91\xd9\x11\x04A\x08Bc\xb3\x...","{'nucl': b""x\x9c5\x91\xcb\x11\x001\x08Bks8\xd8...",{'nucl': b'x\x9c=\x91\xd9\x11\xc3@\x08Ck\xd3\x...,{'nucl': b'x\x9c=Q\xc9\x11\x04A\x08\x8a\x8d\xe...
OG0002935,{'nucl': b'x\x9c=\x92\xdd\r\xc0@\x08\x82g3>\xb...,{'nucl': b'x\x9c=\x92\xdd\r\xc0@\x08\x82g3>\xb...,"{'nucl': b""x\x9cM\x92\xd1\r\x031\x08CgC|x\x01\...",{'nucl': b'x\x9cM\x92\xdb\r\x05A\x08Bk#~\xd8\x...,{'nucl': b'x\x9c=\x92\xdd\r\xc0@\x08\x82g3>\xb...,{'nucl': b'x\x9c=\x92\xdd\r\xc0@\x08\x82g3>\xb...,{'nucl': b'x\x9cM\x92\xdb\r\x05A\x08Bk#~\xd8\x...,{'nucl': b'x\x9c=\x92\xdd\r\xc0@\x08\x82g3>\xb...,"{'nucl': b""x\x9cM\x92\xd1\r\x031\x08CgC|x\x01\..."
OG0002221,{'nucl': b'x\x9c-\x8f\xdb\r\x000\x08\x02g3|\xb...,{'nucl': b'x\x9c-\x8f\xdb\r\x000\x08\x02g3|\xb...,{'nucl': b'x\x9c5\x90\xc1\x11\x00!\x0c\x02kcx\...,{'nucl': b'x\x9c-\x8f\xd1\r\x001\x08Bg#|\xb0\x...,{'nucl': b'x\x9c-\x8f\xdb\r\x000\x08\x02g3|\xb...,{'nucl': b'x\x9c-\x8f\xdb\r\x000\x08\x02g3|\xb...,{'nucl': b'x\x9cU\x8f\xd1\r\xc0@\x08Bg#|\xb0\x...,{'nucl': b'x\x9c-\x8f\xdb\r\x000\x08\x02g3|\xb...,{'nucl': b'x\x9c5\x90\xc1\x11\x00!\x0c\x02kcx\...
OG0002220,{'nucl': b'x\x9c5\x8f\xcb\r\x00A\x08Bk3\x1eh\x...,{'nucl': b'x\x9c5\x8f\xcb\r\x00A\x08Bk3\x1eh\x...,{'nucl': b'x\x9c5\x8f\xdb\r\xc0P\x08Bg#~\xb0\

for k in acc.keys():
    g2ranges = imp_stat.loc[imp_stat['Node'] == k]
    
    g_beg = g2ranges["FA Beg"].values.copy()
    g_end = g2ranges["FA End"].values.copy()
    
    print(k)
    for i in range(len(g_beg)):
        print(set(acc[k][g_beg[i]:g_end[i]]))

In [302]:
# assign improtations to proteins

g_beg = imp_stat["FA Beg"].values.copy()
g_end = imp_stat["FA End"].values.copy()
g_node = imp_stat["Node"].values.copy()

import_acc = []
import_products = []
for i in range(len(g_beg)):
    import_acc.append(set(acc[g_beg[i]:g_end[i]]))
    
    products = []
    for a in import_acc[-1]:
        products.append(zlib.decompress(prot_db.loc[a, g_node[i]]['product']).decode())
    import_products.append(products)
    
imp_stat['Prot-ACC'] = import_acc
imp_stat['Product'] = import_products

In [303]:
imp_stat.to_csv(updated_recombis, index=False)
imp_stat

,Node,Beg,End,FA Beg,FA End,debug_len_must,debug_len_is,Prot-ACC,Product
63,IBSBF_2571,18392,18528,6391,6527,136,136,{OG0000222},[['lauroyl acyltransferase']]
1201,RS_489,24163,24407,8162,8406,244,244,{OG0000226},[['hypothetical protein']]
1339,UW576,24163,24407,8162,8406,244,244,{OG0000226},[['hypothetical protein']]
1202,RS_489,25741,25946,8740,8945,205,205,{OG0000227},[['fructose-bisphosphate aldolase class II']]
1340,UW576,25741,25946,8740,8945,205,205,{OG0000227},[['fructose-bisphosphate aldolase class II']]
...,...,...,...,...,...,...,...,...,...
264,IBSBF_2571,2924360,2925356,1389359,1390355,996,996,{OG0001749},[['sodium:proton antiporter']]
62,RS_489,2939510,2939648,1396509,1396647,138,138,{OG0001757},[['substrate-binding domain-containing protein']]
1338,RS_489,2945986,2946091,1399985,1400090,105,105,{OG0001760},[['ribokinase']]
1476,UW576,2945986,2946091,1399985,1400090,105,105,{OG0001760},[['ribokinase']]


In [304]:
imp_stat = pd.read_csv(updated_recombis)
imp_stat = imp_stat.drop(['Node'], axis=1).drop_duplicates()
imp_stat['Product'].value_counts(normalize=True)[:50].to_csv(top_recombis)

# TODO {'WP_071623447.1'} seq length is less than improtation length?
# TODO enrichment with probabilities 
e.g. numOf(T3 effectors with recombinations)/numOf(all T3 effectors in the genome)

In [305]:
#prot_db.loc["WP_071507848.1","ASM1330693"]

In [306]:
# If split repeat everything!

if SPlIT:
    
    prevL = sum(aln_length)
    
    # in
    recombis = recombis.replace('_cfml', '_cfml2')
    path2tree = path2tree.replace('_cfml', '_cfml2')
    xm_msa_in = xm_msa_in.replace('_s1.xmfa', '_s2.xmfa')
    fa_msa_in = fa_msa_out
    accDB = accDB
    protDB = protDB
    # out
    fa_msa_out = fa_msa_out.replace('_TMP.fasta', '_no_RR.fasta') 
    updated_recombis = updated_recombis.replace('.csv', '2.csv')
    mult_recombis = mult_recombis.replace('.txt', '2.txt')
    top_recombis = top_recombis.replace('.txt', '2.txt')
    
    imp_stat = pd.read_csv(recombis, sep='\t')
    
    for tree in Phylo.parse(path2tree, 'newick'):
        parents = {}
        # iterate through inner nodes 
        for clade in tree.get_nonterminals(order="level"):
            parents[clade.name] = []
            # get leaves for each inner node
            for child in clade.get_terminals():
                parents[clade.name].append(child.name)
                
    
    new_len = len(imp_stat)
    for p in parents:

        is_node = imp_stat["Node"].str.startswith(p)
        num_child = len(parents[p]) 
        print(p,'has',len(imp_stat[is_node]),'recombinations and',num_child,'children')
        new_len = new_len + len(imp_stat[is_node]) * (num_child - 1)

        #imp_stat = imp_stat.append([imp_stat[is_node]]*(num_child-1), ignore_index=True)
        print()

        # for each child: duplicate the parent row
        for child in parents[p]:       
            node_imp = imp_stat[is_node].replace({p:child}) 
            imp_stat = imp_stat.append([node_imp], ignore_index=True)
            is_node = imp_stat["Node"].str.startswith(p)

    # remove the remaining parent rows

    imp_stat = imp_stat.loc[~imp_stat["Node"].str.contains("NODE_")]
    print('DF length after parent nodes expantion =', len(imp_stat), '; Must be =', new_len)
    assert(new_len == len(imp_stat))
    
    # Sort and sum the importations
    imp_stat = imp_stat.sort_values(by=['Beg', 'End'])
    beg = imp_stat["Beg"].values.copy()
    end = imp_stat["End"].values.copy()
    ranges2del = []
    for i in range(len(imp_stat)):
        ranges2del.extend(range(beg[i]-1, end[i]))
    ranges2del = list(dict.fromkeys(ranges2del))
    print('Summed recombination length:', len(ranges2del))


    # # Fasta vs. XMFA positions mapping list
    # 
    # * list of updated positions 
    # * index represent Fasta sequence positions 
    # * values represent XMFA sequence positions 

    # In[14]:


    # parse the genes MSAs into vector of the alignments' lengths
    aln_length = []
    aln = open(xm_msa_in, 'r')

    for l in aln:

        if l.startswith('>'):
            c = 0
        elif l.startswith('='):
            aln_length.append(c-1) # minus one char for the \n symbol
        else:
            c = c + len(l)

    print('Found genes MSAs / LCBs:', len(aln_length))    
    aln.close()


    # In[15]:


    # insert 1000 count after each gene MSA
    index_mapping = list(range(0,aln_length[0]))
    sum_lengths = aln_length[0]

    for i in range(1,len(aln_length)):  
        index_mapping.extend(range(sum_lengths + i*1000, sum_lengths + aln_length[i] + i*1000))
        sum_lengths = sum_lengths + aln_length[i]



    # In[16]:


    print('Fasta length', sum(aln_length), '\nXMFA length', sum(aln_length)+(len(aln_length)-1)*1000)
    # index = fasta file length
    assert(sum(aln_length) == len(index_mapping))
    # values = xmfa file length
    assert(sum(aln_length)+(len(aln_length)-1)*1000 == index_mapping[-1]+1) 


    # # Get the fasta position of each insertion from the xmfa position
    # 
    # + use ranges to improve the running time of .index(i, start, end)

    # In[17]:


    fa_beg = []
    fa_end = []
    previous_beg = 0
    previous_end = 0
    L=len(index_mapping)

    for r_i in range(len(imp_stat)):

        # -1 for 0-start indexing
        r_beg = beg[r_i] - 1
        r_end = end[r_i] - 1
        #print(r_beg, r_end)

        # USE PREVIOUS_BEG FOR BOTH BECAUSE OF THE SORTING ISSUE
        r_beg_index = index_mapping.index(r_beg, previous_beg, L)#r_beg+1)
        r_end_index = index_mapping.index(r_end, previous_beg, L)#r_end+1)

        previous_beg = r_beg_index
        previous_end = r_end_index

        fa_beg.append(r_beg_index)
        fa_end.append(r_end_index)

        #print(r_i,"xmfa - fa:",r_beg_index, r_end_index)

    imp_stat['FA Beg'] = fa_beg
    imp_stat['FA End'] = fa_end


    # In[18]:


    imp_stat['debug_len_must'] = imp_stat['End'] - imp_stat['Beg']
    imp_stat["debug_len_is"] = imp_stat['FA End'] - imp_stat['FA Beg']
    #assert(len(imp_stat.loc[imp_stat['debug_len_must'] != imp_stat['debug_len_is']]) == 0)
    #imp_stat
    imp_stat.loc[imp_stat['debug_len_must'] != imp_stat['debug_len_is']].to_csv(mult_recombis, index=False)


    # In[20]:


    imp_stat.loc[imp_stat['debug_len_must'] != imp_stat['debug_len_is']]


    # # Delete the recombinations from the fasta file 

    # In[30]:


    fasta_out = open(fa_msa_out, 'w')
    num_of_strains = 0
    for record in SeqIO.parse(fa_msa_in, 'fasta'):
        num_of_strains = num_of_strains + 1
        fasta_out.write('>' + record.id + '\n')
        genome_imp_stat = imp_stat.loc[imp_stat['Node'] == record.id]
        beg = genome_imp_stat["FA Beg"].values.copy()
        end = genome_imp_stat["FA End"].values.copy()

        ranges2del = []
        for i in range(len(genome_imp_stat)):
            ranges2del.extend(range(beg[i]-1, end[i]))
        ranges2del = list(dict.fromkeys(ranges2del))

        print('Genome',record.id,'has',len(ranges2del),'recombinations')
        #print(ranges2del)

        print('Number of gaps before',record.seq.count('-'))
        seq = list(record.seq)
        ### prevL - length of the previous splitted xmfa file!!!
        for to_del in ranges2del:
            seq[to_del+prevL] = '-'
        seq = ''.join(seq)
        print('Number of gaps after',seq.count('-'))
        print()
        fasta_out.write(seq + '\n')

    fasta_out.close()


    # # Expand and export the importation_status table

    # In[32]:


    # read Accession Database

    with open(accDB, 'rb') as handle:
        acc = pickle.load(handle)
    acc#.keys()


    # In[33]:


    # read Protein Database

    prot_db = pd.read_pickle(protDB)
    prot_db = prot_db.rename(columns=lambda x: str(x).split('$')[0])


    # In[34]:


    # assign improtations to proteins

    g_beg = imp_stat["FA Beg"].values.copy() + prevL
    g_end = imp_stat["FA End"].values.copy() + prevL
    g_node = imp_stat["Node"].values.copy()

    import_acc = []
    import_products = []
    for i in range(len(g_beg)):
        import_acc.append(set(acc[g_beg[i]:g_end[i]]))

        products = []
        for a in import_acc[-1]:
            products.append(zlib.decompress(prot_db.loc[a, g_node[i]]['product']).decode())
        import_products.append(products)

    imp_stat['Prot-ACC'] = import_acc
    imp_stat['Product'] = import_products


    # In[35]:


    imp_stat.to_csv(updated_recombis, index=False)
    imp_stat


    # In[36]:


    imp_stat = pd.read_csv(updated_recombis)
    imp_stat = imp_stat.drop(['Node'], axis=1).drop_duplicates()
    imp_stat['Product'].value_counts(normalize=True)[:50].to_csv(top_recombis)

    

NODE_17 has 0 recombinations and 9 children

NODE_16 has 0 recombinations and 8 children

NODE_15 has 28 recombinations and 7 children

NODE_14 has 138 recombinations and 5 children

NODE_13 has 82 recombinations and 2 children

NODE_12 has 62 recombinations and 4 children

NODE_11 has 1 recombinations and 3 children

NODE_10 has 0 recombinations and 2 children

DF length after parent nodes expantion = 1756 ; Must be = 1756
Summed recombination length: 118729
Found genes MSAs / LCBs: 1550
Fasta length 1583400 
XMFA length 3132400
Genome Po82 has 44972 recombinations
Number of gaps before 58800
Number of gaps after 103333

Genome UY031 has 0 recombinations
Number of gaps before 11052
Number of gaps after 11052

Genome UW163 has 45002 recombinations
Number of gaps before 56485
Number of gaps after 101101

Genome IBSBF1503 has 45011 recombinations
Number of gaps before 56843
Number of gaps after 101414

Genome RS_488 has 59 recombinations
Number of gaps before 13946
Number of gaps after 1